# Original concept

In [ ]:

#this function will add hydrometeor confihuration to pam2.hydrometeors and 
#append hydrometeor to coordinates of e.g. pam2.profile.lwc
#Note that only the configuration will stored! All the functions
#will be calles later by the forward model! 
pam2.addHydrometeor(
    name = 'snow1', #or None, then str(index)
    index = 0, #or None, then append
    kind = 'snow', #liquid, ice
    nBins = 80,
    sizes = (
        pamtra2.hydrometeors.sizes.logspace, #function/object to call for getting sizes
        funcArgs = {
            Dmin = 1e-6,
            Dmax = 1e-4,
        }
    ),
    psd = (
        pamtra2.hydrometeors.sizeDistributions.exponentialFieldLwc, #function/object to call for getting psd
        funcArgs = {
            temperature : pam2.profile.temperature,
            lwc : pam2.profile.lwc, #test whether hydrodim is included! for all input vars!
            massSizeA : 0.01, #mass size relation required to estimate exponential doistribution from N0
            massSizeB : 1.8,
        },
    ),
    aspectRatio = (
        0.6,
    ),
    mass = (
        pamtra2.hydrometeors.mass.powerLaw, #function/object to call for getting mass
        funcArgs = {
            massSizeA : 0.01,
            massSizeB : 1.8,
        },
    ),
    density = (
        pamtra2.hydrometeors.density.softSphere, #function/object to call for getting density
        funcArgs = {
            minDensity : 100,
#             maxDensity : , defaults to ice!
        },
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.area.powerLaw, #function/object to call for getting crossSectionArea
        funcArgs = {
            areaSizeA : 0.01,
            areaSizeB : 1.8,
        },
    ),
)
#for example, pamtra2.hydrometeors.area.powerLaw will be an object
# which will be called with pamtra2.hydrometeors.area.powerLaw(diameterCenter,
# areaSizeA=areaSizeA,areaSizeB=areaSizeB)



pam2.addHydrometeor(
    name = 'rain1', #or None, then str(index)
    index = 1, #or None, then append
    kind = 'liquid', #liquid, ice
    nBins =40,
    sizes = (
        pamtra2.hydrometeors.sizes.linspace, 
        funcArgs = {
            Dmin = 1e-6,
            Dmax = 1e-4,
        }
    ),
    psd = (
        pamtra2.hydrometeors.sizeDistributions.exponentialLwc, #function/object to call for getting psd
        funcArgs = {
            lwc : pam2.profile.lwc, #test whether hydrodim is included! for all input vars!
            lambd : 10,
        },
    ),
    aspectRatio = (
        pamtra2.hydrometeors.aspectRatio.rainAspectRatioModel,
        funcArgs = {
            sampleKeyword : 10, 
        },
    ),
    mass = (
        pamtra2.hydrometeors.mass.ellipsoid,
    ),
    density = (
        pamtra2.hydrometeors.density.water, # or 1000.
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.area.ellipsoid,
    ),
)

#in case we already have the full spectrum!

pam2.addHydrometeor(
    name = 'ice1', #or None, then str(index)
    index = 1, #or None, then append
    kind = 'liquid', #liquid, ice
    nBins =2,
    sizes = (
        pamtra2.hydrometeors.sizes.monodispers, 
        funcArgs = {
            Dmono = 1e-5,
        },
    ),
    psd = (
        pamtra2.profiles.hydrometeor_size_distribution, #array with discrete n(D) exists, but has len ==0.
    ),
    aspectRatio = (
        1.0,
    ),
    mass = (
        pamtra2.hydrometeors.mass.ellipsoid,
    ),
    density = (
        pamtra2.hydrometeors.density.ice, # or 971.
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.area.ellipsoid,
    ),
)
#now the array is prepared because addHydrometeor got an array as an argumetn for psd
pam2.profiles.hydrometeor_size_distribution.sel(hydrometeor='ice1')[:] = ICE_PSD_FROM_AIRCRAFT


#now also profile.lwc is prepare because we know how many hydrometeors we have:
pam2.profile.lwc[:] = 0.1


# OR EASIER for all of the above: 
pamtra2.importer.profiles.CosmoColumsNetcdf('COSMO_column_20170102.nc') #will take care of hydrometeors and profiles


#kazr is a copy of the pam object with added information about the instrument
kazr = pam2.createInstrument(
    name= 'KAZR'
    kind = 'dopplerRadar',
    #the pamtra2.forwardOperators.spectralRadarSimulator class will contain all the magic and create the object to be retruend
    method = pamtra2.forwardOperators.spectralRadarSimulator, 
    frequencies = [35],
    settings = {
        'nyquistVMax' : 10,
        'nyquistVMin' : -10,
        'nFFT' : 512,
        'K2': 0.92,
        ...
    }
)


#add information about scattering properties
kazr.setHydrometeorScattering(
    'rain1',
    refractiveIndex = (
        pamtra2.refractiveIndex.water, 
        funcArgs = {
            'model' : 'Turner',
        }
    ),
    singleScattering = (
        pamtra2.singleScattering.tmatrix, 
        funcArgs = {
            'cached' : True,
        },
    ),
)

kazr.setHydrometeorScattering(
    'snow1', 
    refractiveIndex = (
        pamtra2.refractiveIndex.snow, 
        funcArgs = {
            'model_mix':'Bruggeman',
            'model_ice':'Matzler_2006',
        }
    ),
    singleScattering = (
        pamtra2.singleScattering.rayleighGans, 
        funcArgs = {
        },
    ),
)

kazr.setHydrometeorScattering(
    'ice1',
    # refractiveIndex = (), For missing descriptions, default values 
    # will be used depending on particle kind!
    # singleScattering = (
    #     pamtra2.singleScattering.mie, 
    # ),
)


hatpro = pam2.createInstrument(
    name= 'Hatpro'
    kind = 'MWR',
    #pamtra2.forwardOperators.RT4 will share a lot of code with pamtra2.forwardOperators.spectralRadarSimulator
    method = pamtra2.forwardOperators.RT4, 
    frequencies = [22.24, 23.04, 23.84, 25.44, 26.24, 27.84, 31.40],
    settings = {
        'bandWidths' : [0.230, 0.230, 0.230, 0.230, 0.230, 0.230, 0.230],
        ....
    }
)

#We can either copy the scattering properties or do new ones!
hatpro.setHydrometeorScattering(
...
)



ceilo = pam2.createInstrument(
    name= 'Ceilo'
    kind = 'Ceilometer',
    method = pamtra2.forwardOperators.Ceilosim, 
    wavelengths = [905],
    settings = {
        'property' : 10,
        ...
    }
)

ceilo.setHydrometeorScattering(
...
)

# OR EASIER: 
WSACR = pamtra.importer.instruments.WSACR(site='Oliktok Point',configuration='20171004')
WSACR.setHydrometeorScattering(
...
)



kazr.run() #this command runs all the functions defined above!
kazr.results.to_netcdf('kazr.nc')

hatpro.run()
hatpro.results.to_netcdf('hatpro.nc')

ceilo.run()
ceilo.results.to_netcdf('ceilo.nc')